### How GPs are implemented in {brms}?

* http://paul-buerkner.github.io/brms/reference/gp.html

**Function signature**

* **...:**	One or more predictors for the GP.
* **by:** A numeric or factor variable of the same length as each predictor. In the numeric vector case, the elements multiply the values returned by the GP. In the factor variable case, a separate GP is fitted for each factor level.
* **k:** Optional number of basis functions for computing approximate GPs. If NA (the default), exact GPs are computed.
* **cov:** Name of the covariance kernel. By default, the exponentiated-quadratic kernel "exp_quad" is used.
* **iso:** A flag to indicate whether an isotropic (`TRUE`; the default) or a non-isotropic GP should be used. In the former case, the same amount of smoothing is applied to all predictors. In the latter case, predictors may have different smoothing. Ignored if only a single predictors is supplied.
* **gr:** Logical; Indicates if auto-grouping should be used (defaults to TRUE). If enabled, observations sharing the same predictor values will be represented by the same latent variable in the GP. This will improve sampling efficiency drastically if the number of unique predictor combinations is small relative to the number of observations.
* **cmc:** Logical; Only relevant if `by` is a factor. If TRUE (the default), cell-mean coding is used for the by-factor, that is one GP per level is estimated. If FALSE, contrast GPs are estimated according to the contrasts set for the by-factor.
* **scale:** Logical; If TRUE (the default), predictors are scaled so that the maximum Euclidean distance between two points is 1. This often improves sampling speed and convergence. Scaling also affects the estimated length-scale parameters in that they resemble those of scaled predictors (not of the original predictors) if scale is TRUE.
* **c:** Numeric value only used in approximate GPs. Defines the multiplicative constant of the predictors' range over which predictions should be computed. A good default could be c = 5/4 but we are still working on providing better recommendations.



**Questions**

* Why would we want to use the `by` argument with a numeric variable?
    * Potential answer: It's like an interaction between a GP and a numerical variable. This also makes sense when `by` is categorical.
* How easy is to switch from an isotropic to a non-isotropic GP in PyMC?
    * If I remember correctly the process is quite manual
* How to set priors for the parameters of the GP?
    * Lengthscale
    * Standard deviation
    * ...?
* How to allow for different mean and covariance functions?
    * What is the mean function in {brms}? It doesn't seem to allow for different mean functions
* `brms` currently supports only the exponentiated-quadratic kernel. Is there a fundamental reason to do so, that we may be missing?
* Does `gr` make sense for non-approximated GPs?
    * First, what does it refer to? I think it means it combines many GPs into one?
    * Second, does it make sense for non-approximated GPs? I think the answer is yes.
    * I suspect this is not the same as sharing the basis when using HSGP
    * Check with Bill

**Comments**

* I like having a `cov` argument where we pass the name of the covariance kernel.
    * How would one pass kernel specific parameters? (which parameters one may want to tweak?)
    * The con is the formula will be longer and harder to read. Let's keep thinking about better alternatives that don't make the formula too long.
* The `gp()` function is used for both exact and approximated GPs (via HSGP).
    * I think it's better to both `gp()` and `hsgp()` in our case.

**Resources and Examples**

* https://bookdown.org/ajkurz/Statistical_Rethinking_recoded/adventures-in-covariance.html
* https://github.com/paul-buerkner/brms/issues/412
* https://discourse.mc-stan.org/t/auto-grouping-option-in-brms-gp/11154
* https://projecteuclid.org/journals/bayesian-analysis/volume-5/issue-1/Bayesian-functional-ANOVA-modeling-using-Gaussian-process-prior-distributions/10.1214/10-BA505.full
* https://discourse.mc-stan.org/t/auto-grouping-of-latent-variable-in-gaussian-process/14866/5
* https://discourse.mc-stan.org/t/incorporating-distance-matrices-into-gaussian-process-in-brms/17978/12
* https://discourse.mc-stan.org/t/recovering-lscale-in-a-two-dimensional-gaussian-process-regression/8458
    * Experiment to recover parameter values
* https://github.com/mike-lawrence/hgpr
    * Hierarchical GP in Stan
* https://discourse.mc-stan.org/t/sharing-a-to-me-neat-parameterization-for-gaussian-processes/21701/2
    * Alternative parametrization to avoid problems with variance estimation
* https://mc-stan.org/users/documentation/case-studies/icar_stan.html
    * Similar than above, but a tutorial with spatial data

In [3]:
from formulae.terms.call_resolver import get_function_from_module
from formulae import design_matrices

import numpy as np
import pandas as pd

import pymc as pm

df = pd.DataFrame({"x": np.arange(10)})

In [2]:
GP_KERNELS = {            
    "ExpQuad": pm.gp.cov.ExpQuad,
    "Matern32": pm.gp.cov.Matern32,
    "Matern52": pm.gp.cov.Matern52,
}

In [15]:
class HSGP:
    def __call__(self, *x, m, L=None, c=None, by=None, cov="ExpQuad", drop_first=True):
        self.by = by
        self.m = m
        self.L = L
        self.c = c
        self.by = by
        self.cov = cov
        self.drop_first = drop_first
        return np.column_stack(x)

hsgp = HSGP()

In [20]:
m = [10]
dm = design_matrices("1 + hsgp(x, m=m)", df)

In [21]:
np.array(dm.common)

array([[1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [1, 6],
       [1, 7],
       [1, 8],
       [1, 9]])

In [25]:
from formulae.terms.call import Call
from formulae.terms.call_resolver import get_function_from_module


def is_single_component(term):
    return hasattr(term, "components") and len(term.components) == 1


def extract_first_component(term):
    return term.components[0]


def is_call_component(component):
    return isinstance(component, Call)


def is_caller_of_instance(caller, kind):
    function = get_function_from_module(caller.call.callee, caller.env)
    return isinstance(function, kind)


def is_hsgp_term(term):
    if not is_single_component(term):
        return False
    component = extract_first_component(term)
    if not is_call_component(component):
        return False
    return is_caller_of_instance(component, HSGP)


def extract_hsgp_kwargs(term):
    component = extract_first_component(term)
    caller = get_function_from_module(component.call.callee, component.env)
    return caller.__dict__

In [28]:
for name, term in dm.common.terms.items():
    if is_hsgp_term(term):
        print(name)
        print(extract_hsgp_kwargs(term))

hsgp(x, m = m)
{'by': None, 'm': [10], 'L': None, 'c': None, 'cov': 'ExpQuad', 'drop_first': True}


In [ ]:
class HSGPTerm:
    def __init__(self, term, prior, prefix=None):
        self.term = term
        self.prior = prior # I think this may need to be dictionary
        self.data = np.squeeze(term.data)
        self.prefix = prefix
        self.hsgp_attrs = extract_hsgp_kwargs(term)

    @property
    def term(self):
        return self._term

    @term.setter
    def term(self, value):
        #assert isinstance(value, formulae.terms.terms.Term)
        self._term = value

    @property
    def m(self):
        return self.hsgp_attrs["m"]

    @property
    def L(self):
        return self.hsgp_attrs["L"]

    @property
    def c(self):
        return self.hsgp_attrs["c"]

    @property
    def cov(self):
        return self.hsgp_attrs["cov"]

    @property
    def drop_first(self):
        return self.hsgp_attrs["drop_first"]

    @property
    def prior(self):
        return self._prior

    @prior.setter
    def prior(self, value):
        # assert isinstance(value, VALID_PRIORS), f"Prior must be one of {VALID_PRIORS}"
        self._prior = value

---

* $\phi$ is of shape $(n, m)$
* $\text{psd}$ is of shape $(m, )$
* $\beta$ is of shape $(m, )$
* $f$ is of shape $(n, )$

In [ ]:
with pm.Model():    
    eta = pm.Exponential("eta", lam=2.0)
    ell = pm.InverseGamma("ell", mu=10, sigma=1)
    cov = eta**2 * pm.gp.cov.ExpQuad(1, ls=ell)
    gp = pm.gp.HSGP(m=[10], c=2.0, cov_func=cov)
    
    X = pm.MutableData("X", x[:, None])
    phi, sqrt_psd = gp.prior_cosmponents(X=X)

    # Non-centered
    beta = pm.Normal("beta", size=sqrt_psd.size)
    f = pm.Deterministic("f", phi @ (beta * sqrt_psd))

    # Centered
    #beta = pm.Normal("beta", sigma=sqrt_psd)
    #f = pm.Deterministic("f", phi @ beta)